# Entregable

Trabajas en una agencia que se dedica a organizar viajes a personas, tu cargo
específicamente es analista de datos del área de clientes VIP.

Ha llegado a la agencia un cantante famoso, conocido en España, que desea que le
ubiquéis un buen alojamiento que le permita disfrutar del sol para sus próximas
vacaciones de verano que serán a finales del mes de julio del 2025.

Este cliente quiere descansar porque necesita prepararse para una gira de conciertos que
dará en septiembre y por ahora quiere alejarse del foco mediático todo lo que pueda
durante sus vacaciones, no le gustaría llamar la atención durante esos días ya que estará
con sus 2 padres, sus 2 hermanos pequeños, su novia y los padres de su novia.

El cliente no tiene claro el destino al que quiere ir, solo sabe que tiene que ser en alguna
playa. Tu trabajo será encontrar la propiedad en la que el cliente y su familia se alojarán
durante sus vacaciones de verano teniendo en cuenta las exigencias del cliente.

Descargar las librerias para utilizar en este proyecto:

In [2]:
from sqlalchemy import create_engine
import sqlalchemy
import mysql.connector as mysql
import pandas as pd

In [3]:
# Parámetros de conexión
user     = 'root'
password = ''
db = ''
sqlServer = 'localhost' # debería ser IP o DNS (Domain Name System) del servidor de la base de datos

# Creamos string de conexión
# connStr = "mysql+mysqlconnector://" + user + ":" + password + "@" + sqlServer # +  "/"  + db
# connStr = "mysql+mysqlconnector://{}:{}@{}?charset=utf8".format(user,password,sqlServer)
connStr = "mysql+mysqlconnector://{}:{}@{}/{}?charset=utf8".format(user,password,sqlServer,"NUCLIO")


"""
connect_string = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(DB_USER, DB_PASS, DB_HOST, DB_PORT, DATABASE)
"""

# print de string de conexión
print (connStr)

# Conectamos con el servidor
conn = create_engine(connStr).connect() 
engine = create_engine(connStr)

mysql+mysqlconnector://root:@localhost/NUCLIO?charset=utf8


In [4]:
alojamientos = pd.read_excel('ALOJAMIENTO.xlsx')
precios = pd.read_excel('PRECIO.xlsx')
puntuacion = pd.read_excel('PUNTUACION.xlsx')
ubicacion = pd.read_excel('UBICACION.xlsx')

In [39]:
# alojamientos.to_sql('alojamientos', conn)
# precios.to_sql('precios', conn)
# puntuacion.to_sql('puntuacion', conn)
# ubicacion.to_sql('ubicacion_2', conn)

20

In [7]:
# Prueba de query
sSQL="""
    select *
    from alojamientos  
    limit 5   
"""
pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,NOMBRE,CANT_BANYOS,CANT_HAB,SALON,TERRAZA,AIREC_ACOND,PISCINA,SUPERFICIE
0,0,ALOJ001,Villa Sol del Mar,2,3,SI,SI,SI,SI,120
1,1,ALOJ002,Apartamento Las Palmeras,1,2,SI,NO,SI,NO,75
2,2,ALOJ003,Cabaña El Refugio,1,1,NO,SI,NO,NO,45
3,3,ALOJ004,Piso Centro Histórico,2,3,SI,SI,SI,NO,90
4,4,ALOJ005,Estudio Playa Blanca,1,1,NO,NO,SI,NO,40


In [31]:
ubicacion.head()

,CODIGO_ALOJAMIENTO,PAIS,CIUDAD,DIST_METRO,DIST_CENTRO
0,ALOJ001,Portugal,Liboa,1.0,2.0
1,ALOJ002,España,Madrid,0.5,0.8
2,ALOJ003,España,Barcelona,0.9,0.4
3,ALOJ004,Francia,Marsella,3.0,4.0
4,ALOJ005,Portugal,Oporto,4.0,3.0


In [41]:
# Prueba de query
sSQL="""
    select count(*)
    from ubicacion_2 u
"""
pd.read_sql(sSQL, conn)

,count(*)
0,20


## Primer paso

Crear la query de cada uno de las exigencias del cliente. Se debe crear una
query para cada punto. (7 queries en total)

- El cliente exige terraza y aire acondicionado, pero no quiere piscina.
- No se alojará en estudios que no tengan salón ni en propiedades con solo un baño
- La propiedad debe tener como mínimo 80m2
- Como el cliente odia el ruido, no quiere alojarse en propiedades a menos de 1 km
de una estación de metro ni a menos de 2 km del centro.
- El rango permitido por el cliente para pagar es entre 1.500€ y 2.000€ la noche.
- Como en anteriores ocasiones ha tenido problemas de intentos de estafas con
otras agencias, el cliente pide que el % de reserva no sea muy alto, pero tampoco
tan bajo (porque le genera inseguridad) por lo que propone un 25%
- Quiere que el alojamiento tenga más de 4,5 puntos de confianza y que la agencia
que lo lleva tenga más de 4 puntos

**Query:** El cliente exige terraza y aire acondicionado, pero no quiere piscina.

In [9]:
# El cliente exige terraza y aire acondicionado, pero no quiere piscina.
sSQL="""
    select 
        CODIGO_ALOJAMIENTO, NOMBRE, TERRAZA, PISCINA, AIREC_ACOND
    from alojamientos  
    where TERRAZA = 'SI' AND PISCINA = 'NO' AND AIREC_ACOND = 'SI'
"""
pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,TERRAZA,PISCINA,AIREC_ACOND
0,ALOJ004,Piso Centro Histórico,SI,NO,SI
1,ALOJ010,Ático del Sol,SI,NO,SI
2,ALOJ015,Dúplex Las Rocas,SI,NO,SI
3,ALOJ018,Piso Parque Central,SI,NO,SI


**Query:** No se alojará en estudios que no tengan salón ni en propiedades con solo un baño

In [10]:
# No se alojará en estudios que no tengan salón ni en propiedades con solo un baño
sSQL="""
    select 
        CODIGO_ALOJAMIENTO, NOMBRE, SALON, CANT_BANYOS
    from alojamientos  
    where SALON = 'SI' AND CANT_BANYOS > 1
"""
pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,SALON,CANT_BANYOS
0,ALOJ001,Villa Sol del Mar,SI,2
1,ALOJ004,Piso Centro Histórico,SI,2
2,ALOJ006,Chalet Vista al Lago,SI,3
3,ALOJ007,Dúplex Jardines del Sur,SI,2
4,ALOJ009,Casa Rústica Montaña,SI,2
5,ALOJ010,Ático del Sol,SI,3
6,ALOJ012,Villa Palmeras,SI,2
7,ALOJ014,Casa del Mar,SI,3
8,ALOJ015,Dúplex Las Rocas,SI,3
9,ALOJ017,Chalet Jardín Secreto,SI,3


**Query:** La propiedad debe tener como mínimo 80m2.


In [11]:
# No se alojará en estudios que no tengan salón ni en propiedades con solo un baño
sSQL="""
    select 
        CODIGO_ALOJAMIENTO, NOMBRE, SUPERFICIE
    from alojamientos  
    where SUPERFICIE >= 80
    order by SUPERFICIE DESC
"""
pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,SUPERFICIE
0,ALOJ017,Chalet Jardín Secreto,180
1,ALOJ014,Casa del Mar,160
2,ALOJ006,Chalet Vista al Lago,150
3,ALOJ007,Dúplex Jardines del Sur,130
4,ALOJ001,Villa Sol del Mar,120
5,ALOJ015,Dúplex Las Rocas,120
6,ALOJ012,Villa Palmeras,110
7,ALOJ018,Piso Parque Central,100
8,ALOJ019,Casa Río Claro,95
9,ALOJ004,Piso Centro Histórico,90


**Query:** Como el cliente odia el ruido, no quiere alojarse en propiedades a menos de 1 km
de una estación de metro ni a menos de 2 km del centro.

In [42]:
sSQL="""
    select 
        u.CODIGO_ALOJAMIENTO, a.NOMBRE, DIST_METRO, DIST_CENTRO
    from ubicacion_2  u
    LEFT JOIN alojamientos a
        on a.CODIGO_ALOJAMIENTO = u.CODIGO_ALOJAMIENTO
    WHERE DIST_METRO >= 1 AND DIST_CENTRO >= 2
"""
pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,DIST_METRO,DIST_CENTRO
0,ALOJ001,Villa Sol del Mar,1.0,2.0
1,ALOJ004,Piso Centro Histórico,3.0,4.0
2,ALOJ005,Estudio Playa Blanca,4.0,3.0
3,ALOJ006,Chalet Vista al Lago,2.0,2.0
4,ALOJ007,Dúplex Jardines del Sur,1.0,3.0
5,ALOJ008,Loft Urbano,2.0,3.0
6,ALOJ009,Casa Rústica Montaña,2.0,4.0
7,ALOJ010,Ático del Sol,2.0,5.0
8,ALOJ015,Dúplex Las Rocas,2.0,4.0
9,ALOJ016,Estudio Urbano,2.0,2.0


**Query:** El rango permitido por el cliente para pagar es entre 1.500€ y 2.000€ la noche.


In [13]:
precios.head()

,CODIGO_ALOJAMIENTO,PRECIO_ALQUILER,RESERVA,PORCENTAJE_RESERVA
0,ALOJ001,1800,SI,100
1,ALOJ002,780,SI,25
2,ALOJ003,800,NO,0
3,ALOJ004,1600,SI,25
4,ALOJ005,900,NO,0


In [18]:
sSQL="""
    select 
        p.CODIGO_ALOJAMIENTO, a.NOMBRE, p.PRECIO_ALQUILER precio
    from precios  p
    LEFT JOIN alojamientos a
        on a.CODIGO_ALOJAMIENTO = p.CODIGO_ALOJAMIENTO
   where p.PRECIO_ALQUILER BETWEEN 1500 and 2000
"""
pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,precio
0,ALOJ001,Villa Sol del Mar,1800
1,ALOJ004,Piso Centro Histórico,1600
2,ALOJ008,Loft Urbano,1500
3,ALOJ009,Casa Rústica Montaña,1600
4,ALOJ010,Ático del Sol,1750
5,ALOJ011,Cabaña Bosque Encantado,1825
6,ALOJ012,Villa Palmeras,1750
7,ALOJ013,Apartamento Moderno,1652
8,ALOJ014,Casa del Mar,1941
9,ALOJ015,Dúplex Las Rocas,1800


**Query:** Como en anteriores ocasiones ha tenido problemas de intentos de estafas con
otras agencias, el cliente pide que el % de reserva no sea muy alto, pero tampoco
tan bajo (porque le genera inseguridad) por lo que propone un 25%

In [19]:
sSQL="""
    select 
        p.CODIGO_ALOJAMIENTO, a.NOMBRE, p.porcentaje_reserva 
    from precios  p
    LEFT JOIN alojamientos a
        on a.CODIGO_ALOJAMIENTO = p.CODIGO_ALOJAMIENTO
   where porcentaje_reserva = 25
"""
pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,porcentaje_reserva
0,ALOJ002,Apartamento Las Palmeras,25
1,ALOJ004,Piso Centro Histórico,25
2,ALOJ008,Loft Urbano,25
3,ALOJ010,Ático del Sol,25
4,ALOJ015,Dúplex Las Rocas,25
5,ALOJ018,Piso Parque Central,25


**Query:** Quiere que el alojamiento tenga más de 4,5 puntos de confianza y que la agencia
que lo lleva tenga más de 4 puntos

In [21]:
puntuacion.head()

,CODIGO_ALOJAMIENTO,PUNTOS,AGENCIA,PUNTOS_AGENCIA
0,ALOJ001,5.0,GlobalHome,4.4
1,ALOJ002,4.2,AndesRenta,4.1
2,ALOJ003,4.1,AlquilaFacil,4.0
3,ALOJ004,4.6,CaribeRent,4.6
4,ALOJ005,4.4,RentaTop,3.8


In [25]:
# Quiere que el alojamiento tenga más de 4,5 puntos de confianza y que la agencia
# que lo lleva tenga más de 4 puntos

sSQL="""
    select 
        p.CODIGO_ALOJAMIENTO, a.NOMBRE, PUNTOS, PUNTOS_AGENCIA
    from puntuacion p
    left join alojamientos a
        on p.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
    where PUNTOS_AGENCIA > 4 AND PUNTOS > 4.5
""" 

pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,PUNTOS,PUNTOS_AGENCIA
0,ALOJ001,Villa Sol del Mar,5.0,4.4
1,ALOJ004,Piso Centro Histórico,4.6,4.6
2,ALOJ007,Dúplex Jardines del Sur,4.9,4.5
3,ALOJ008,Loft Urbano,4.8,4.3
4,ALOJ009,Casa Rústica Montaña,4.7,4.8
5,ALOJ010,Ático del Sol,4.8,4.9
6,ALOJ014,Casa del Mar,4.7,4.5
7,ALOJ015,Dúplex Las Rocas,4.9,4.7
8,ALOJ017,Chalet Jardín Secreto,4.9,4.4
9,ALOJ018,Piso Parque Central,4.7,4.7


## Segundo Paso

Crear en una única query la combinación de los filtros anteriores para hallar las propiedades que finalmente cumplen con todas las exigencias del cliente.

In [57]:
sSQL="""
with cte_terraza as (
        select 
            CODIGO_ALOJAMIENTO, NOMBRE, TERRAZA, PISCINA, AIREC_ACOND
        from alojamientos  
        where TERRAZA = 'SI' AND PISCINA = 'NO' AND AIREC_ACOND = 'SI'
    ),
    cte_salon_banyo as (
        select 
            CODIGO_ALOJAMIENTO, NOMBRE, SALON, CANT_BANYOS
        from alojamientos  
        where SALON = 'SI' AND CANT_BANYOS > 1
    ),
    cte_80 as (
        select 
            CODIGO_ALOJAMIENTO, NOMBRE, SUPERFICIE
        from alojamientos  
        where SUPERFICIE >= 80
        order by SUPERFICIE DESC
    ),
    cte_no_en_centro as (
        select 
            u.CODIGO_ALOJAMIENTO, u.PAIS, u.CIUDAD, a.NOMBRE, DIST_METRO, DIST_CENTRO
        from ubicacion_2  u
        LEFT JOIN alojamientos a
            on a.CODIGO_ALOJAMIENTO = u.CODIGO_ALOJAMIENTO
        WHERE DIST_METRO >= 1 AND DIST_CENTRO >= 2
    ),
    cte_precios as (
            select 
            p.CODIGO_ALOJAMIENTO, a.NOMBRE, p.PRECIO_ALQUILER precio
        from precios  p
        LEFT JOIN alojamientos a
            on a.CODIGO_ALOJAMIENTO = p.CODIGO_ALOJAMIENTO
    where p.PRECIO_ALQUILER BETWEEN 1500 and 2000
    ),
    cte_reserva as (
            select 
            p.CODIGO_ALOJAMIENTO, a.NOMBRE, p.porcentaje_reserva 
        from precios  p
        LEFT JOIN alojamientos a
            on a.CODIGO_ALOJAMIENTO = p.CODIGO_ALOJAMIENTO
    where porcentaje_reserva = 25
    ),
    cte_puntos as (
        select 
            p.CODIGO_ALOJAMIENTO, a.NOMBRE, PUNTOS, PUNTOS_AGENCIA
        from puntuacion p
        left join alojamientos a
            on p.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
        where PUNTOS_AGENCIA > 4 AND PUNTOS > 4.5
    )
    select 
        a.CODIGO_ALOJAMIENTO, 
        a.nombre, 
        n.PAIS,
        n.CIUDAD,
        a.terraza,
        a.piscina,
        a.airec_acond,
        b.salon,
        b.cant_banyos,
        e.superficie,
        n.dist_metro,
        n.dist_centro,
        p.precio,
        r.porcentaje_reserva,
        pu.puntos,
        pu.puntos_agencia
    from alojamientos a
    inner join cte_terraza t
        on t.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
    inner join cte_salon_banyo b
        on a.CODIGO_ALOJAMIENTO = b.CODIGO_ALOJAMIENTO
    inner join cte_80 e
        on a.CODIGO_ALOJAMIENTO = e.CODIGO_ALOJAMIENTO    
    inner join cte_no_en_centro n
        on a.CODIGO_ALOJAMIENTO = n.CODIGO_ALOJAMIENTO 
    inner join cte_precios p 
        on a.CODIGO_ALOJAMIENTO = p.CODIGO_ALOJAMIENTO     
    inner join cte_reserva r 
        on a.CODIGO_ALOJAMIENTO = r.CODIGO_ALOJAMIENTO 
    inner join cte_puntos pu 
        on a.CODIGO_ALOJAMIENTO = pu.CODIGO_ALOJAMIENTO     
"""

pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,nombre,PAIS,CIUDAD,terraza,piscina,airec_acond,SALON,CANT_BANYOS,SUPERFICIE,DIST_METRO,DIST_CENTRO,precio,porcentaje_reserva,PUNTOS,PUNTOS_AGENCIA
0,ALOJ004,Piso Centro Histórico,Francia,Marsella,SI,NO,SI,SI,2,90,3.0,4.0,1600,25,4.6,4.6
1,ALOJ010,Ático del Sol,España,Bilbao,SI,NO,SI,SI,3,88,2.0,5.0,1750,25,4.8,4.9
2,ALOJ015,Dúplex Las Rocas,España,Málaga,SI,NO,SI,SI,3,120,2.0,4.0,1800,25,4.9,4.7
3,ALOJ018,Piso Parque Central,España,Ibiza,SI,NO,SI,SI,2,100,2.0,4.0,1900,25,4.7,4.7


## Tercer Paso

El cliente no tiene claro el destino al que quiere ir, solo sabe que tiene que ser en __**alguna playa**.__ 
Tu trabajo será encontrar la propiedad en la que el cliente y su familia se alojarán
durante sus vacaciones de verano teniendo en cuenta las exigencias del cliente.

Este cliente quiere descansar porque necesita prepararse para una gira de conciertos que
dará en septiembre y por ahora quiere alejarse del foco mediático todo lo que pueda
durante sus vacaciones, **no le gustaría llamar la atención durante esos días ya que estará con sus 2 padres, sus 2 hermanos pequeños, su novia y los padres de su novia.**

In [53]:
ubicacion['CIUDAD'].unique

<bound method Series.unique of 0         Liboa
1        Madrid
2     Barcelona
3      Marsella
4        Oporto
5         Paris
6      Toulouse
7     Barcelona
8        Madrid
9        Bilbao
10        Milan
11         Roma
12      Sevilla
13     Cerdegna
14       Málaga
15         Roma
16        Paris
17        Ibiza
18        Lagos
19      Madeira
Name: CIUDAD, dtype: object>

In [63]:
# ALOJ015
sSQL="""
    select 
        u.*, a.*, p.*, pr.*
    from alojamientos  a
    left join ubicacion_2 u
        on u.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
    left join puntuacion p
        on p.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
    left join precios pr
        on pr.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
    where u.CODIGO_ALOJAMIENTO in ('ALOJ015','ALOJ018')

"""
pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,PAIS,CIUDAD,DIST_METRO,DIST_CENTRO,index,CODIGO_ALOJAMIENTO,NOMBRE,CANT_BANYOS,...,index,CODIGO_ALOJAMIENTO,PUNTOS,AGENCIA,PUNTOS_AGENCIA,index,CODIGO_ALOJAMIENTO,PRECIO_ALQUILER,RESERVA,PORCENTAJE_RESERVA
0,14,ALOJ015,España,Málaga,2.0,4.0,14,ALOJ015,Dúplex Las Rocas,3,...,14,ALOJ015,4.9,EasyStay Valencia,4.7,14,ALOJ015,1800,SI,25
1,17,ALOJ018,España,Ibiza,2.0,4.0,17,ALOJ018,Piso Parque Central,2,...,17,ALOJ018,4.7,MendozaRooms,4.7,17,ALOJ018,1900,SI,25


Recomendaría **ALOJ015** porque:
- Es a lado de la playa, que el cliente busca.
- Tiene más baños, para su familia que se queda con el cantante.
- Una puntuación mejor.
- Precio un poco menos, si eso es lo que le importe.

In [64]:
# ALOJ015
sSQL="""
    select 
        u.*, a.*, p.*, pr.*
    from alojamientos  a
    left join ubicacion_2 u
        on u.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
    left join puntuacion p
        on p.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
    left join precios pr
        on pr.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
    where u.CODIGO_ALOJAMIENTO in ('ALOJ015')

"""
pd.read_sql(sSQL, conn)

,index,CODIGO_ALOJAMIENTO,PAIS,CIUDAD,DIST_METRO,DIST_CENTRO,index,CODIGO_ALOJAMIENTO,NOMBRE,CANT_BANYOS,...,index,CODIGO_ALOJAMIENTO,PUNTOS,AGENCIA,PUNTOS_AGENCIA,index,CODIGO_ALOJAMIENTO,PRECIO_ALQUILER,RESERVA,PORCENTAJE_RESERVA
0,14,ALOJ015,España,Málaga,2.0,4.0,14,ALOJ015,Dúplex Las Rocas,3,...,14,ALOJ015,4.9,EasyStay Valencia,4.7,14,ALOJ015,1800,SI,25


# SEGUNDA PARTE
Responde las siguientes preguntas (con su respectiva query)

- ¿Cuál es la suma total de metros cuadrados de los alojamientos que tienen piscina?
- ¿Cuál es el alojamiento con mayor superficie?
- ¿Cuál es el alojamiento con menor superficie?
- ¿Cuántos alojamientos tienen salón?
- ¿Cuántos alojamientos tienen terraza?
- ¿Cuántos alojamientos hay por cada país según nuestra tabla de UBICACIÓN?
- ¿Cuál es el promedio de precios de alquiler de todos los alojamientos según la
tabla PRECIO?
- ¿Cuáles son las 3 agencias que tienen la mayor puntuación de agencia (campo a
usar: PUNTOS_AGENCIA) según la tabla PUNTUACION?
- Muestra los alojamientos (código y nombre) que: Contengan la palabra ‘Piso’ en su
nombre o que tengan piscina y aire acondicionado, pero no pasen los 150 metros de superficie o que tengan más de un baño
- Muestra una consulta que muestre el código de alojamiento, el nombre, el precio y una columna donde veamos el precio del alquiler con el símbolo del euro al final de cada importe, esta nueva columna se debe llama PRECIO_MODIF
- Muestra una columna sobre la tabla PRECIO que se llame CATEGORIA_PRECIO y que indique la palabra ‘Bajo’ si el precio está entre 780 y 1000, ‘Medio’ si el precio está entre 1000 y 1700 y ‘Alto’ si es mayor a 1700

In [66]:
# ¿Cuál es la suma total de metros cuadrados de los alojamientos que # tienen piscina?

sSQL="""
    select 
        sum(SUPERFICIE) as total_superficie
    from alojamientos  a
    WHERE PISCINA = 'SI'
"""
pd.read_sql(sSQL, conn)

,total_superficie
0,850.0


In [71]:
# ¿Cuál es el alojamiento con mayor superficie?

sSQL="""
    select 
        codigo_alojamiento,
        nombre,
        superficie as max_superficie
    from alojamientos  a
    order by max_superficie desc
    limit 1
"""
pd.read_sql(sSQL, conn)


,codigo_alojamiento,nombre,max_superficie
0,ALOJ017,Chalet Jardín Secreto,180


In [75]:
# ¿Cuál es el alojamiento con menor superficie?

sSQL="""
    select 
        codigo_alojamiento,
        nombre,
        superficie as min_superficie
    from alojamientos  a
    order by min_superficie asc
    limit 1
"""
pd.read_sql(sSQL, conn)


,codigo_alojamiento,nombre,min_superficie
0,ALOJ016,Estudio Urbano,35


In [76]:
# ¿Cuántos alojamientos tienen salón?

sSQL="""
    select 
        count(*) as con_salon
    from alojamientos  a
    where SALON = 'SI'
"""
pd.read_sql(sSQL, conn)


,con_salon
0,16


In [77]:
# ¿Cuántos alojamientos tienen terraza?

sSQL="""
    select 
        count(*) as con_terraza
    from alojamientos  a
    where terraza = 'SI'
"""
pd.read_sql(sSQL, conn)

,con_terraza
0,14


In [79]:
# ¿Cuántos alojamientos hay por cada país según nuestra tabla de UBICACIÓN?

sSQL="""
    select 
        pais,
        count(*) as alojamientos
    from ubicacion_2
group by pais
"""
pd.read_sql(sSQL, conn)

,pais,alojamientos
0,Portugal,4
1,España,8
2,Francia,4
3,Italia,4


In [82]:
# ¿Cuál es el promedio de precios de alquiler de todos los alojamientos según la tabla PRECIO?

sSQL="""
    select 
        avg(PRECIO_ALQUILER) as avg_price
    from precios
"""
pd.read_sql(sSQL, conn)

,avg_price
0,1568.4


In [89]:
# ¿Cuáles son las 3 agencias que tienen la mayor puntuación de agencia (campo a usar: PUNTOS_AGENCIA) según la tabla PUNTUACION?

sSQL="""
    select 
        agencia,
        puntos_agencia
    from puntuacion
    order by puntos_agencia desc
    limit 3
"""
pd.read_sql(sSQL, conn)

,agencia,puntos_agencia
0,MalagaTurismo,4.9
1,VivaStay,4.9
2,UYAlquila,4.9


In [92]:
# Muestra los alojamientos (código y nombre) que: Contengan la palabra ‘Piso’ en su nombre o que tengan piscina y aire acondicionado, pero no pasen los 150 metros o que tengan más de un baño.

sSQL="""
    select 
        CODIGO_ALOJAMIENTO,
        NOMBRE
    from alojamientos
    where lower(NOMBRE) LIKE '%piso%' or
    (piscina = 'SI' and AIREC_ACOND = 'SI' and SUPERFICIE < 150) or CANT_BANYOS > 1
"""
pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE
0,ALOJ001,Villa Sol del Mar
1,ALOJ004,Piso Centro Histórico
2,ALOJ006,Chalet Vista al Lago
3,ALOJ007,Dúplex Jardines del Sur
4,ALOJ009,Casa Rústica Montaña
5,ALOJ010,Ático del Sol
6,ALOJ012,Villa Palmeras
7,ALOJ014,Casa del Mar
8,ALOJ015,Dúplex Las Rocas
9,ALOJ017,Chalet Jardín Secreto


In [94]:
# Muestra una consulta que muestre el código de alojamiento, el nombre, el precio y una columna donde veamos el precio del alquiler con el símbolo del euro al final de cada importe, esta nueva columna se debe llama PRECIO_MODIF

sSQL="""
    select 
        a.CODIGO_ALOJAMIENTO,
        NOMBRE,
        PRECIO_ALQUILER,
        concat(PRECIO_ALQUILER,'€') as PRECIO_MODIF
    from alojamientos a
    left join precios p
        on p.CODIGO_ALOJAMIENTO = a.CODIGO_ALOJAMIENTO
"""
pd.read_sql(sSQL, conn)

,CODIGO_ALOJAMIENTO,NOMBRE,PRECIO_ALQUILER,PRECIO_MODIF
0,ALOJ001,Villa Sol del Mar,1800,1800€
1,ALOJ002,Apartamento Las Palmeras,780,780€
2,ALOJ003,Cabaña El Refugio,800,800€
3,ALOJ004,Piso Centro Histórico,1600,1600€
4,ALOJ005,Estudio Playa Blanca,900,900€
5,ALOJ006,Chalet Vista al Lago,1200,1200€
6,ALOJ007,Dúplex Jardines del Sur,1300,1300€
7,ALOJ008,Loft Urbano,1500,1500€
8,ALOJ009,Casa Rústica Montaña,1600,1600€
9,ALOJ010,Ático del Sol,1750,1750€


In [95]:
# Muestra una columna sobre la tabla PRECIO que se llame CATEGORIA_PRECIO y que indique la palabra ‘Bajo’ si el precio está entre 780 y 1000, ‘Medio’ si el precio está entre 1000 y 1700 y ‘Alto’ si es mayor a 1700

sSQL="""
    select 
        codigo_alojamiento,
        case
            when PRECIO_ALQUILER between 780 and 1000 then 'Bajo'
            when PRECIO_ALQUILER between 1000 and 1700 then 'Medio'
            when PRECIO_ALQUILER > 1700 then 'Alto' 
        end as CATEGORIA_PRECIO
    from precios
"""
pd.read_sql(sSQL, conn)

,codigo_alojamiento,CATEGORIA_PRECIO
0,ALOJ001,Alto
1,ALOJ002,Bajo
2,ALOJ003,Bajo
3,ALOJ004,Medio
4,ALOJ005,Bajo
5,ALOJ006,Medio
6,ALOJ007,Medio
7,ALOJ008,Medio
8,ALOJ009,Medio
9,ALOJ010,Alto
